In [ ]:
def simulate(self,
                mc_fun,
                ebno_dbs,
                batch_size,
                max_mc_iter,
                legend="",
                add_ber=True,
                add_bler=False,
                soft_estimates=False,
                num_target_bit_errors=None,
                num_target_block_errors=None,
                early_stop=True,
                graph_mode=None,
                add_results=True,
                forward_keyboard_interrupt=True,
                show_fig=True,
                verbose=True):
    """Simulate BER/BLER curves for given Keras model and saves the results.

    Internally calls :class:`sionna.utils.sim_ber`.

    Input
    -----
    mc_fun:
        Callable that yields the transmitted bits `b` and the
        receiver's estimate `b_hat` for a given ``batch_size`` and
        ``ebno_db``. If ``soft_estimates`` is True, b_hat interpreted as
        logit.

    ebno_dbs: ndarray of floats
        SNR points to be evaluated.

    batch_size: tf.int32
        Batch-size for evaluation.

    max_mc_iter: int
        Max. number of Monte-Carlo iterations per SNR point.

    legend: str
        Name to appear in legend.

    add_ber: bool
        Defaults to True. Indicate if BER should be added to plot.

    add_bler: bool
        Defaults to False. Indicate if BLER should be added
        to plot.

    soft_estimates: bool
        A boolean, defaults to False. If True, ``b_hat``
        is interpreted as logit and additional hard-decision is applied
        internally.

    num_target_bit_errors: int
        Target number of bit errors per SNR point until the simulation
        stops.

    num_target_block_errors: int
        Target number of block errors per SNR point until the simulation
        stops.

    early_stop: bool
        A boolean defaults to True. If True, the simulation stops after the
        first error-free SNR point (i.e., no error occurred after
        ``max_mc_iter`` Monte-Carlo iterations).

    graph_mode: One of ["graph", "xla"], str
        A string describing the execution mode of ``mc_fun``.
        Defaults to `None`. In this case, ``mc_fun`` is executed as is.

    add_results: bool
        Defaults to True. If True, the simulation results will be appended
        to the internal list of results.

    show_fig: bool
        Defaults to True. If True, a BER figure will be plotted.

    verbose: bool
        A boolean defaults to True. If True, the current progress will be
        printed.

    forward_keyboard_interrupt: bool
        A boolean defaults to True. If False, `KeyboardInterrupts` will be
        catched internally and not forwarded (e.g., will not stop outer
        loops). If False, the simulation ends and returns the intermediate
        simulation results.

    Output
    ------
    (ber, bler):
        Tuple:

    ber: float
        The simulated bit-error rate.

    bler: float
        The simulated block-error rate.
    """

    ber, bler = sim_ber(mc_fun,
                        ebno_dbs,
                        batch_size,
                        soft_estimates=soft_estimates,
                        max_mc_iter=max_mc_iter,
                        num_target_bit_errors=num_target_bit_errors,
                        num_target_block_errors=num_target_block_errors,
                        early_stop=early_stop,
                        graph_mode=graph_mode,
                        verbose=verbose,
                    forward_keyboard_interrupt=forward_keyboard_interrupt)

    if add_ber:
        self._bers += [ber]
        self._snrs +=  [ebno_dbs]
        self._legends += [legend]
        self._is_bler += [False]

    if add_bler:
        self._bers += [bler]
        self._snrs +=  [ebno_dbs]
        self._legends += [legend + " (BLER)"]
        self._is_bler += [True]

    if show_fig:
        self()

    # remove current curve if add_results=False
    if add_results is False:
        if add_bler:
            self.remove(-1)
        if add_ber:
            self.remove(-1)

    return ber, bler
